In [1]:
# import libraries
import numpy as np
import cv2
import sys 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load images
img_left = cv2.imread("left.jpeg")
img_right = cv2.imread("right.jpeg")
left_gray= cv2.cvtColor(img_left,cv2.COLOR_BGR2GRAY)
right_gray= cv2.cvtColor(img_right,cv2.COLOR_BGR2GRAY)


In [4]:
# detect keypoints,descriptors
orb = cv2.ORB_create()
kp_left, des_left = orb.detectAndCompute(img_left,None)
kp_right, des_right = orb.detectAndCompute(img_right,None)

# show detect keypoints,descriptors on images 
cv2.imshow("kp_left",cv2.drawKeypoints(left_gray, kp_left, None, (255, 0, 0)))
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("kp_right",cv2.drawKeypoints(right_gray,kp_right,None, (255, 0, 0)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
bf = cv2.DescriptorMatcher_create("BruteForce")
matches = bf.knnMatch(des_left, des_right,k=2)
matches

[[<DMatch 000001D37E3A0050>, <DMatch 000001D309036390>],
 [<DMatch 000001D309036890>, <DMatch 000001D309036850>],
 [<DMatch 000001D309036570>, <DMatch 000001D309036770>],
 [<DMatch 000001D3090368D0>, <DMatch 000001D309036870>],
 [<DMatch 000001D309036830>, <DMatch 000001D309036790>],
 [<DMatch 000001D3090362F0>, <DMatch 000001D3090364F0>],
 [<DMatch 000001D3090368F0>, <DMatch 000001D309036910>],
 [<DMatch 000001D309036930>, <DMatch 000001D309036950>],
 [<DMatch 000001D309036970>, <DMatch 000001D309036990>],
 [<DMatch 000001D3090369B0>, <DMatch 000001D3090369D0>],
 [<DMatch 000001D3090369F0>, <DMatch 000001D309036A10>],
 [<DMatch 000001D309036A30>, <DMatch 000001D309036A50>],
 [<DMatch 000001D309036A70>, <DMatch 000001D309036A90>],
 [<DMatch 000001D309036AB0>, <DMatch 000001D309036AD0>],
 [<DMatch 000001D309036AF0>, <DMatch 000001D309036B10>],
 [<DMatch 000001D309036B30>, <DMatch 000001D309036B50>],
 [<DMatch 000001D309036B70>, <DMatch 000001D309036B90>],
 [<DMatch 000001D309036BB0>, <D

In [6]:
# print the descriptor of a first point.
print("des_left:", des_left[0]);
print("des_right:", des_right[0]);


des_left: [191  73 107 234 118 211  79 212  85 192 244 138 123   6 183  48 126 249
  23 254 233 182  60 174 243 201 164 128 117 187  98 248]
des_right: [ 91 120 175  60  13  86 205  63 225 202  70  69 120 252  23 126 196 100
 249 207 205 197 183 233 227 127 222 136  14 244 114 124]


In [8]:
# less distance is better match
good = []
for match1, match2 in matches:
    # ıf match 1 distance is less than 75% of match 2 distance
    #then descriptor was a dood match, keep it
    if match1.distance < 0.75 * match2.distance:
        good.append(match1)

In [9]:
# show best matches with 75%
cv2.imshow("key",cv2.drawKeypoints(left_gray, [kp_left[m.queryIdx] for m in good], None, (255, 0, 255)))
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imshow("key",cv2.drawKeypoints(right_gray, [kp_left[m.queryIdx] for m in good], None, (255, 0, 255)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
print(len(good));
print(len(matches))

47
500


In [11]:
def draw_matches(img_left, kp_left, img_right, kp_right, sel_matches):
    # get the dimetions of images
    left_row, left_col = img_left.shape
    right_row, right_col = img_right.shape

    # To show the result get ready imageboard as concatenate two image 
    # get max row it will be the height it should be one not addition of two  row value
    match_img = np.zeros((max([left_row,  right_row]), left_col+right_col, 3), dtype='uint8')
    match_img[:left_row, :left_col, :] = np.dstack([img_left, img_left, img_left])
    match_img[: right_row, left_col:left_col+right_col, :] = np.dstack([img_right, img_right, img_right])
    
    #queryIdx refers to img_left_index and trainIdx refers to img_right_index
    for m in sel_matches:
        #index of the descriptor in the list of  query descriptors 
        img_left_index = m.queryIdx
        #index of the descriptor in the list of train descriptors 
        img_right_index = m.trainIdx
        
        # get the key points according to index
        (xleft, yleft) = kp_left[img_left_index].pt
        (xright, yright) = kp_right[img_right_index].pt
        
        # draw them with circle function 
        target1 = (int(xleft),int(yleft))
        target2 = (int(xright)+left_col,int(yright))
        cv2.circle(match_img, target1, 4, (0, 255, 255), 1)
        cv2.circle(match_img, target2, 4, (0, 255, 255), 1)
        
        # draw a single line for each matching. point  to point         
        cv2.line(match_img, target1, target2, (0, 255, 255), 1)
    
    return match_img

In [12]:
match_img = draw_matches(left_gray, kp_left, right_gray, kp_right, good)
cv2.imshow("good matches",match_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
def warpImages(img_left, img_right, homography):
    
    # get height and weight values
    rowl, coll = img_left.shape[:2]
    rowr, colr = img_right.shape[:2]

    # left to right - corner coordinates
    left = np.float32([[0,0], [0, rowl],[coll, rowl], [coll, 0]]).reshape(-1, 1,2)
    right = np.float32([[0,0], [0,rowr], [colr,rowr], [colr,0]]).reshape(-1,1,2)

    # To apply homography we need wrap perspective. Calculate the transformation matrix
    right = cv2.perspectiveTransform(right, homography)
        
    points = np.concatenate((left,right), axis=0)

    # get min max coordinates 
    min_p = points.min(axis=0).ravel() # search for one dim
    max_p = points.max(axis=0).ravel()
    [minx, miny] = np.int32(min_p - 0.5)
    [maxx, maxy] = np.int32(max_p + 0.5)
  
    block = [-minx,-miny]
    hom_translation = np.array([[1, 0, block[0]], [0, 1, block[1]], [0, 0, 1]])

    final_img = cv2.warpPerspective(img_right, hom_translation.dot(homography), (maxx-minx, maxy-miny))
    final_img[block[1]:rowl+block[1], block[0]:coll+block[0]] = img_left

    return final_img

In [15]:
# convert keypoints for findHomography()
#img_left_index = kp_left[match.queryIdx].pt
#img_right_index = kp_right[match.trainIdx].pt 

# get them according to index 
left_match=[ kp_left[match.queryIdx].pt for match in good]
right_match=[ kp_right[match.trainIdx].pt for match in good] 
# convert for homography
fixed = np.float32(left_match).reshape(-1,1,2)
stable = np.float32(right_match).reshape(-1,1,2)

# calculated homography matrix
homo, other = cv2.findHomography(fixed, stable, cv2.RANSAC,5.0)
#  stitch images together 
result = warpImages(img_right, img_left, homo)

cv2.imshow("result",result)
cv2.waitKey(0)
cv2.destroyAllWindows()